In [31]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard

from PIL import Image

In [32]:
def load_and_preprocess_dataset(dataset_folder):
    image_data = []
    person_folders = sorted(os.listdir(dataset_folder))

    for person_folder in person_folders:
        person_path = os.path.join(dataset_folder, person_folder)
        if os.path.isdir(person_path):
            image_paths = sorted(os.listdir(person_path))
            for image_name in image_paths:
                image_path = os.path.join(person_path, image_name)
                img = load_img(image_path, target_size=(128, 128))
                img = img_to_array(img) / 255.0
                try:
                    image = Image.open(image_path)
                except OSError:
                    continue
                image_data.append(img)

    return np.array(image_data)

In [33]:
def generate_triplets(data, num_triplets=10000):
    triplets = []
    for _ in range(num_triplets):
        anchor_idx = np.random.randint(0, data.shape[0])
        positive_idx = np.random.randint(0, data.shape[0])
        while positive_idx == anchor_idx:
            positive_idx = np.random.randint(0, data.shape[0])
        negative_idx = np.random.randint(0, data.shape[0])
        while negative_idx == anchor_idx or negative_idx == positive_idx:
            negative_idx = np.random.randint(0, data.shape[0])
        triplets.append([data[anchor_idx], data[positive_idx], data[negative_idx]])
    return np.array(triplets)

In [34]:
def create_base_network(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu')
    ])
    return model

In [35]:
def generate_test_triplets(data):
    triplets = []
    for _ in range(data.shape[0]):
        anchor_idx = np.random.randint(0, data.shape[0])
        positive_idx = np.random.randint(0, data.shape[0])
        while positive_idx == anchor_idx:
            positive_idx = np.random.randint(0, data.shape[0])
        negative_idx = np.random.randint(0, data.shape[0])
        while negative_idx == anchor_idx or negative_idx == positive_idx:
            negative_idx = np.random.randint(0, data.shape[0])
        triplets.append([data[anchor_idx], data[positive_idx], data[negative_idx]])
    return np.array(triplets)

In [36]:
def create_and_compile_model(input_shape):
    base_network = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
    ])

    anchor_input = tf.keras.layers.Input(input_shape, name='anchor_input')
    positive_input = tf.keras.layers.Input(input_shape, name='positive_input')
    negative_input = tf.keras.layers.Input(input_shape, name='negative_input')

    encoded_anchor = base_network(anchor_input)
    encoded_positive = base_network(positive_input)
    encoded_negative = base_network(negative_input)

    merged_output = tf.keras.layers.concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1)

    # Normalize the output embedding vectors
    merged_output = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=-1))(merged_output)

    model = tf.keras.models.Model(inputs=[anchor_input, positive_input, negative_input], outputs=merged_output)

    # Compile the model
    model.compile(optimizer='adam', loss=triplet_loss)

    return model

In [37]:
dataset_folder = '1/lfw-deepfunneled/lfw-deepfunneled/'
image_data = load_and_preprocess_dataset(dataset_folder)

# Generate triplets for training
num_triplets = 10000
triplets_train = generate_triplets(image_data, num_triplets=num_triplets)


In [38]:
train_data, val_data = train_test_split(triplets_train, test_size=0.2)

dummy_labels_train = np.zeros((train_data.shape[0], 1))
dummy_labels_val = np.zeros((val_data.shape[0], 1))

In [39]:
def triplet_loss(y_true, y_pred, alpha=0.2):
    anchor, positive, negative = y_pred[:, :128], y_pred[:, 128:256], y_pred[:, 256:]
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    basic_loss = pos_dist - neg_dist + alpha
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    return loss

In [40]:
def load_and_preprocess_test_dataset(test_folder):
    test_image_data = []
    test_labels = []

    person_folders = sorted(os.listdir(test_folder))
    
    for person_folder in person_folders:
        person_path = os.path.join(test_folder, person_folder)
        if os.path.isdir(person_path):
            image_paths = sorted(os.listdir(person_path))
            for image_name in image_paths:
                image_path = os.path.join(person_path, image_name)
                img = load_img(image_path, target_size=(128, 128))
                img = img_to_array(img) / 255.0
                test_image_data.append(img)
                test_labels.append(person_folder)  # Assuming folder names are the labels

    return np.array(test_image_data), np.array(test_labels)

In [41]:
input_shape = (128, 128, 3)
model = create_and_compile_model(input_shape)

In [42]:
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

In [43]:
model.fit(
    [train_data[:, 0], train_data[:, 1], train_data[:, 2]],
    dummy_labels_train,
    validation_data=([val_data[:, 0], val_data[:, 1], val_data[:, 2]], dummy_labels_val),
    epochs=10,
    batch_size=64,
    callbacks=[tensorboard_callback]  # TensorBoard callback ekleniyor
)


Epoch 1/10
125/125 [==============================] - 30s 232ms/step - loss: 12.7830 - val_loss: 12.7387
Epoch 2/10
125/125 [==============================] - 22s 176ms/step - loss: 12.7776 - val_loss: 12.7415
Epoch 3/10
125/125 [==============================] - 16s 126ms/step - loss: 12.8007 - val_loss: 12.7297
Epoch 4/10
125/125 [==============================] - 19s 154ms/step - loss: 12.7516 - val_loss: 12.7286
Epoch 5/10
125/125 [==============================] - 17s 134ms/step - loss: 12.7689 - val_loss: 12.7442
Epoch 6/10
125/125 [==============================] - 15s 120ms/step - loss: 12.7346 - val_loss: 12.7313
Epoch 7/10
125/125 [==============================] - 14s 116ms/step - loss: 12.6925 - val_loss: 12.7296
Epoch 8/10
125/125 [==============================] - 13s 105ms/step - loss: 12.8033 - val_loss: 12.7374
Epoch 9/10
125/125 [==============================] - 15s 122ms/step - loss: 12.7464 - val_loss: 12.7269
Epoch 10/10
125/125 [==============================] - 

In [44]:
test_folder = '1/lfw-deepfunneled/test/'
test_image_data, test_labels = load_and_preprocess_test_dataset(test_folder)

# Generate triplets for testing
test_triplets = generate_test_triplets(test_image_data)

dummy_labels_test = np.zeros((test_triplets.shape[0], 1))
# Evaluate the model on the test set
model.evaluate([test_triplets[:, 0], test_triplets[:, 1], test_triplets[:, 2]], dummy_labels_test)

110/110 [==============================] - 1s 11ms/step - loss: 6.3809


6.380923271179199

In [48]:
%load_ext tensorboard
%tensorboard --logdir=logs

In [45]:
# Load and preprocess the input image
input_image_path = 'C:/Users/enver/Desktop/giris/11.jpg'
input_img = load_img(input_image_path, target_size=(128, 128))
input_img = img_to_array(input_img) / 255.0
input_img = np.expand_dims(input_img, axis=0)  # Add batch dimension

# Extract embeddings for the input image
input_embedding = model.get_layer('sequential')(input_img)  # 'sequential' is the name of your base network

# Calculate distances
distances = []
for i in range(test_image_data.shape[0]):
    test_embedding = model.get_layer('sequential')(np.expand_dims(test_image_data[i], axis=0))
    distance = np.linalg.norm(input_embedding - test_embedding)
    distances.append(distance)

# Convert distances to numpy array for sorting
distances = np.array(distances)

# Sort and select top k indices
k = 5  # You can adjust k as needed
top_k_indices = np.argsort(distances)[:k]

# Display the top k similar images
import matplotlib.pyplot as plt

for index in top_k_indices:
    similar_image = test_image_data[index]
    plt.imshow(similar_image)
    plt.title(f'Distance: {distances[index]:.4f}')
    plt.show()


ValueError: No such layer: sequential. Existing layers are: ['anchor_input', 'positive_input', 'negative_input', 'sequential_2', 'concatenate_1', 'lambda'].